In [ ]:
pip install cfbd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.0/242.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.7
    Uninstalling pydantic-2.11.7:
      Successfully uninstalled pydantic-2.11.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.23 which is incompatible.
google-genai 1.34.0 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.23 which is incompatible.
gradio 5.44.1 requires pydantic<2.12,>=2.0, but you have pydantic 1.10.23 which is incompatible.
google-adk 1.13.0 requires pydantic<3.0.0

In [ ]:
import os
from getpass import getpass
import cfbd
import pandas as pd

token = os.getenv("CFBD_API_KEY") or os.getenv("BEARER_TOKEN") #r98LboBL/RroLtYFMtLfNQncW28IQKFaFWG9sUyF2FfyFVaUjMFiDgAxfRE6L9l1
if not token:
    token = getpass("Enter your CFBD API key: ").strip()

cfg = cfbd.Configuration()
cfg.access_token = token

with cfbd.ApiClient(cfg) as api_client:
    games_api = cfbd.GamesApi(api_client)
    games = games_api.get_games(year=2015)
    rows = [g.to_dict() if hasattr(g, "to_dict") else dict(g) for g in games]
    df_games_2015 = pd.DataFrame(rows)
    print(df_games_2015.head())

Enter your CFBD API key: ··········
          id  season  week          seasonType                 startDate  \
0  400795501    2015     1  SeasonType.REGULAR 2015-08-29 19:30:00+00:00   
1  400763593    2015     1  SeasonType.REGULAR 2015-09-03 22:00:00+00:00   
2  400603840    2015     1  SeasonType.REGULAR 2015-09-03 22:00:00+00:00   
3  400795766    2015     1  SeasonType.REGULAR 2015-09-03 23:00:00+00:00   
4  400799864    2015     1  SeasonType.REGULAR 2015-09-03 23:00:00+00:00   

   startTimeTBD  completed  neutralSite  conferenceGame  venueId  ...  \
0         False       True        False           False     3987  ...   
1         False       True        False           False     3652  ...   
2         False       True         True           False     3628  ...   
3         False       True        False           False     3909  ...   
4         False       True        False           False     3629  ...   

  highlights  attendance homePostgameWinProbability homePregameElo  

In [ ]:
import os
from getpass import getpass
import cfbd
import pandas as pd
from cfbd.rest import ApiException

token = os.getenv("CFBD_API_KEY") or os.getenv("BEARER_TOKEN")
if not token:
    token = getpass("Enter your CFBD API key: ").strip()

cfg = cfbd.Configuration()

cfg.api_key["Authorization"] = token
cfg.api_key_prefix["Authorization"] = "Bearer"

cfg.access_token = token


years = list(range(2014, 2025 + 1))
all_rows = []

with cfbd.ApiClient(cfg) as api_client:
    games_api = cfbd.GamesApi(api_client)
    for yr in years:
        try:
            games = games_api.get_games(year=yr)
            for g in games:
                rec = g.to_dict() if hasattr(g, "to_dict") else dict(g)
                rec.setdefault("season", yr)
                all_rows.append(rec)
            print(f"Year {yr}: {len(games)} games")
        except ApiException as e:
            print(f"⚠️  Skipping {yr} due to API error: {e}")


df_games = pd.json_normalize(all_rows, sep="_")
if "season" in df_games.columns:
    sort_cols = ["season"]
    if "start_date" in df_games.columns:
        sort_cols.append("start_date")
    df_games = df_games.sort_values(sort_cols).reset_index(drop=True)

print("Combined df_games shape:", df_games.shape)
display(df_games.head())

OUTDIR = r"C:\Users\barry\Downloads"
os.makedirs(OUTDIR, exist_ok=True)
outfile = os.path.join(OUTDIR, "cfbd_games_2014_2025.csv")
df_games.to_csv(outfile, index=False, encoding="utf-8-sig")
print(f"✅ Wrote {len(df_games):,} rows to {outfile}")

Enter your CFBD API key: ··········
Year 2014: 1586 games
Year 2015: 1538 games
Year 2016: 1549 games
Year 2017: 1551 games
Year 2018: 1556 games
Year 2019: 1623 games
Year 2020: 1125 games
Year 2021: 2454 games
Year 2022: 3705 games
Year 2023: 3734 games
Year 2024: 3801 games
Year 2025: 3651 games
Combined df_games shape: (27873, 33)


,id,season,week,seasonType,startDate,startTimeTBD,completed,neutralSite,conferenceGame,venueId,...,highlights,attendance,homePostgameWinProbability,homePregameElo,homePostgameElo,awayPostgameWinProbability,awayPregameElo,awayPostgameElo,excitementIndex,notes
0,400559171,2014,1,SeasonType.REGULAR,2014-08-28 23:30:00+00:00,False,True,True,False,3758.0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
1,400558005,2014,1,SeasonType.REGULAR,2014-08-28 23:00:00+00:00,False,True,False,False,3788.0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,400557454,2014,1,SeasonType.REGULAR,2014-08-28 23:00:00+00:00,False,True,False,False,3963.0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
3,400557394,2014,1,SeasonType.REGULAR,2014-08-28 23:00:00+00:00,False,True,False,False,3658.0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,400558114,2014,1,SeasonType.REGULAR,2014-08-28 23:00:00+00:00,False,True,False,False,3629.0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


✅ Wrote 27,873 rows to C:\Users\barry\Downloads/cfbd_games_2014_2025.csv
